In [1]:
from ast import literal_eval
import pandas as pd
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
res = pd.read_csv('match_results.csv')

In [3]:
for col in ['w1_or', 'w2_or', 'w3_or', 'w4_or', 
            'w1_nr', 'w2_nr', 'w3_nr', 'w4_nr', 
            'l1_or', 'l2_or', 'l3_or', 'l4_or', 
            'l1_nr', 'l2_nr', 'l3_nr', 'l4_nr', 
           ]:
    res[col].fillna(0, inplace=True)
    res[col] = res[col].astype(int)
    

In [4]:
start = pd.read_csv('match_starts.csv')

In [5]:
start = start.drop_duplicates(subset=['match_id'])

In [6]:
combined = res.merge(start, on='match_id', how='inner')

In [7]:
def w_players(row):
    if row['team_a_won']:
        players = literal_eval(row['team_a_players'])
    else:
        players = literal_eval(row['team_b_players'])
    players = [str(i) for i in players]
    players = players + [''] * (4 - len(players))
    players = '|'.join(players)
    return players


In [8]:
def l_players(row):
    if row['team_a_won']:
        players = literal_eval(row['team_b_players'])
    else:
        players = literal_eval(row['team_a_players'])
    players = [str(i) for i in players]
    players = players + [''] * (4 - len(players))
    players = '|'.join(players)
    return players


In [9]:
combined['winners'] = combined.apply(w_players, axis=1)
combined['losers'] = combined.apply(l_players, axis=1)

In [10]:
combined.head()

,match_id,w1,w2,w3,w4,l1,l2,l3,l4,w1_or,...,l3_nr,l4_nr,team_a_won,team_b_won,date,map,team_a_players,team_b_players,winners,losers
0,791864,Water,SaraSaiyan,ForeRuthless,adirath,gravity1075,MrWrong,Savar Loon,NewHaveScouts,1860,...,1076,746,False,True,2023-11-25,Arena (Random civs),"[211286360714117120, 470492953983582219, 70079...","[716142389902704691, 699611550423711766, 84933...",716142389902704691|699611550423711766|84933446...,211286360714117120|470492953983582219|70079298...
1,791810,addu,wArDaDdY,Ju$TK!Dd!nG,ThirtiethBog219,SaraSaiyan,ForeRuthless,THUNDER,NewHaveScouts,2523,...,1074,765,True,False,2023-11-25,4 lakes (Random civs),"[657452537506562069, 512487301717295104, 71479...","[699611550423711766, 849334461773774928, 72356...",657452537506562069|512487301717295104|71479330...,699611550423711766|849334461773774928|72356799...
2,791768,gravity1075,Water,ForeRuthless,adirath,addu,THUNDER,Avenger,Ju$TK!Dd!nG,1951,...,1012,917,False,True,2023-11-25,Golden Pit,"[657452537506562069, 723567999784583239, 22387...","[211286360714117120, 716142389902704691, 84933...",211286360714117120|716142389902704691|84933446...,657452537506562069|723567999784583239|22387552...
3,791697,SaraSaiyan,THUNDER,lucky23,NewHaveScouts,Water,wArDaDdY,Avenger,ThirtiethBog219,1448,...,1025,470,True,False,2023-11-25,Ghost Lake,"[699611550423711766, 723567999784583239, 35857...","[716142389902704691, 512487301717295104, 22387...",699611550423711766|723567999784583239|35857002...,716142389902704691|512487301717295104|22387552...
4,791666,addu,Afterlife,CHICKEN,Aragornn,| Royal,Sephiroth,ForeRuthless,adirath,2529,...,1346,566,False,True,2023-11-25,Ghost Lake,"[725758030221934692, 439339450770653185, 84933...","[657452537506562069, 542430914836168744, 69894...",657452537506562069|542430914836168744|69894350...,725758030221934692|439339450770653185|84933446...


In [11]:
from collections import defaultdict

In [12]:
player_map = defaultdict(list)

In [13]:
def map_players(row):
    winners = [int('0' + i) for i in row['winners'].split('|')]
    losers = [int('0' + i) for i in row['losers'].split('|')]
    w1, w2, w3, w4 = row['w1'], row['w2'], row['w3'], row['w4']
    l1, l2, l3, l4 = row['l1'], row['l2'], row['l3'], row['l4']
    for a, b in zip(winners, [w1, w2, w3, w4]):
        player_map[a].append(b)
    
    for a, b in zip(losers, [l1, l2, l3, l4]):
        player_map[a].append(b)


In [14]:
combined.apply(map_players, axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
5964    None
5965    None
5966    None
5967    None
5968    None
Length: 5969, dtype: object

In [15]:
player_map_final = {}
for k, v in player_map.items():
    player_map[k] = Counter(v).most_common()[0][0]

In [16]:
len(player_map)

154

In [17]:
def id_to_name(ids):
    x = []
    for i in ids.split('|'):
        try:
            x.append(player_map[int(i)])
        except:
            x.append('')
    return '|'.join(x)
combined['winners'] = combined['winners'].apply(id_to_name)
combined['losers'] = combined['losers'].apply(id_to_name)

In [18]:
combined.drop(['w1', 'w2', 'w3', 'w4', 'l1', 'l2', 'l3', 'l4', 
               'team_a_won', 'team_b_won', 
               'team_a_players', 'team_b_players'], 
              axis=1, inplace=True)

In [19]:
combined[['w1', 'w2', 'w3', 'w4']] = combined['winners'].str.split('|', expand=True)
combined[['l1', 'l2', 'l3', 'l4']] = combined['losers'].str.split('|', expand=True)
                                                                  

In [20]:
combined.drop(['winners', 'losers'], axis=1, inplace=True)

In [21]:
combined['year_month'] = combined['date'].apply(lambda x: x[:7])

In [22]:
combined = combined[['match_id', 'date', 'year_month', 'map', 
                     'w1', 'w2', 'w3', 'w4', 
                     'l1', 'l2', 'l3', 'l4', 
                     'w1_or', 'w2_or', 'w3_or', 'w4_or', 
                     'w1_nr', 'w2_nr', 'w3_nr', 'w4_nr', 
                     'l1_or', 'l2_or', 'l3_or', 'l4_or', 
                     'l1_nr', 'l2_nr', 'l3_nr', 'l4_nr'
                    ]]

In [23]:
combined

,match_id,date,year_month,map,w1,w2,w3,w4,l1,l2,...,w3_nr,w4_nr,l1_or,l2_or,l3_or,l4_or,l1_nr,l2_nr,l3_nr,l4_nr
0,791864,2023-11-25,2023-11,Arena (Random civs),Water,Saracen,ForeRuthless,adirath,गुरुत्वाकर्षण,MrWrong,...,1359,598,1964,1407,1095,765,1945,1388,1076,746
1,791810,2023-11-25,2023-11,4 lakes (Random civs),addu,wArDaDdY,Just🅱an,ThirtiethBog219,Saracen,ForeRuthless,...,936,269,1478,1359,1093,784,1459,1340,1074,765
2,791768,2023-11-25,2023-11,Golden Pit,गुरुत्वाकर्षण,Water,ForeRuthless,adirath,addu,THUNDER,...,1359,579,2536,1106,1025,930,2523,1093,1012,917
3,791697,2023-11-25,2023-11,Ghost Lake,Saracen,THUNDER,Lucky23,newisyou,Water,wArDaDdY,...,1106,784,1877,1071,1055,500,1847,1041,1025,470
4,791666,2023-11-25,2023-11,Ghost Lake,addu,Afterlife,CHICKEN,Aragornn,Royal,Sephiroth,...,882,550,1410,1398,1353,573,1403,1391,1346,566
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5964,394835,2022-01-19,2022-01,African Clearing,AgentP,BoseDK,wArDaDdY,Simba,Thiru,TalentButNoFarm,...,1255,941,1787,1655,1194,877,1773,1641,1180,863
5965,394824,2022-01-19,2022-01,Golden Pit,Thiru,Mirror,TheLivi,BraveHeart,BoseDK,Himansu,...,1191,778,1400,1303,1252,870,1389,1286,1241,859
5966,394816,2022-01-19,2022-01,Arena,SMR,Thiru,BraveHeart,Bondulkar,TalentButNoFarm,Himansu,...,767,767,1668,1322,1318,1265,1655,1303,1305,1252
5967,394792,2022-01-19,2022-01,African Clearing,Morty,Thiru,shrp,CrazyGuY,SMR,Afterlife,...,1268,1202,2335,1320,1185,756,2333,1318,1183,754


In [24]:
def fix_map_name(map_):
    map_ = map_.lower()
    random_civs = False
    if 'random civ' in map_:
        random_civs = True
    map_name = map_.split('(')[0].split('-')[0].strip().title()
    if random_civs:
        map_name += ' (Random civs)'
    return map_name
    

In [25]:
def fix_map_only_name(map_):
    return map_.lower().split('(')[0].split('-')[0].strip().title()
    

In [26]:
combined['map'] = combined['map'].apply(fix_map_name)
combined['map_only'] = combined['map'].apply(fix_map_only_name)

In [27]:
combined.to_csv('matches.csv', index=False)